# Recommandation de films via la factorisation matricielle

# 1. Introduction

Dans cet atelier, nous proposons d'implémenter un système de recommandation basé sur la factorisation matricielle. Nous utiliserons la base de données <a href="https://grouplens.org/datasets/movielens/">MovieLens</a> afin d'entraîner nos modèles, mener certaines expériences et comparer nos résultats avec d'autres types d'architectures.


<!-- #region id="pIG0GWqkz8BC" -->

## 1.1 Installation des librairies

Avant de commencer, nous devons nous assurer d'installer les librairies nécessaires pour le tutoriel à l'aide de pip. Pour ce faire, exécutez la cellule suivante en la sélectionnant et en cliquant `shift`+`Enter`. Ceci peut prendre quelques minutes.

In [ ]:
!rm -rf RS-Workshop
!git clone https://github.com/davidberger2785/RS-Workshop

Afin de vous assurer que l'installation ait eu lieu, importez toutes les libraries et modules dont nous nous servirons pour cet atelier en exécutant la prochaine cellule.

In [ ]:
import numpy as np
import pandas as pd

# Visualisation des données
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

# Fonctions maison
sys.path += ['RS-Workshop/Tutoriels - En/']
import utilities as utl

Nous avons également écrit quelques fonctions passe-partout que nous avons regroupées dans la librairie utilities. En fait, ces différentes fonctions existent fort probablement déjà en python, mais nous en ignorons simplement l'existence...

## 1.2 Objectif

De façon générale, l'objectif d'un système de recommandation est, comme son nom l'indique, d'effectuer des recommandations personnalisées à chacun des utilisateurs. Idéalement, ces recommandations devront être bonnes, bien que ce concept puisse rapidement devenir flou. Contrairement à d'autres tâches en apprentissage automatique, telle la reconnaissance d'images de chats ou la prédiction du cours d'une action en bourse, effectuer des recommandations de manière à aider un utilisateur est d'autant plus complexe que ce problème est plus ou moins bien défini. Cherchons-nous à présenter à un utilisateur précis des suggestions le confortant dans ses choix antérieurs? Ou enconre, voulons-nous lui présenter des suggestions complémentaires ou totalement indépendantes des items précédemment considérés? Enfin, tenterons-nous plutôt de lui présenter des items auxquels il n'a pas encore été exposé? Chacune des ces options sont légitimes et pourront être modélisées. Sans perte de généralités, le schéma ci-dessous modélise simplement la problématique des systèmes de recommandation sous l'angle de l'apprentissage automatique.

![title](https://github.com/davidberger2785/RS-Workshop/blob/master/Images/High_level_1.png?raw=1)



N'empêche, dans le cadre de cet atelier et en considérant le contexte dans lequel nous sommes plongés, soit la suggestion de films comme le font Netflix ou Amazon Prime, nous pouvons réduire le problème à une tâche relativement simple: recommander des films que l'utilisateur va aimer en fonction de ses intérêts passés. Afin de mener à bien cette tâche, nous utiliserons l'ensemble des préférences des usagers, certaines variables sociodémographiques associées de même que certaines caractéristiques des films. Enfin, nous pouvons rafiner le schéma ainsi:

![title](https://github.com/davidberger2785/RS-Workshop/blob/master/Images/High_level_2.png?raw=1)


## 1.3 Jeu de données - MoviesLens 100k

Les données utilisées consistent ici en plus ou moins 100 000 évaluations de films effectuées par 943 utilisateurs et où un ensemble de 1 682 films étaient disponibles en visionnement. En plus des 100 000 évaluations à notre disposition, nous avons des informations complémentaires liées à chacun des usagers de même qu'à chacun des films.

En somme, nous allons utiliser un total de trois jeux de données afin de mener à bien nos analyses soit:

* Users : contenant des informations associées aux caractéristiques des utilisateurs,
* Movies : contenant des informations associées aux caractéristiques des films en visionnement,
* Ratings : contenant l'ensemble des 100 000 évaluations effectuées par les utilisateurs.

Nous utiliserons la librairie <a href="https://pandas.pydata.org/">Pandas</a> pour télécharger la base de données.


### 1.3.1 Users: Importation et prétraitement des données

In [ ]:
# Téléchargement des données
ROOT_DIR = 'RS-Workshop/'
DATA_DIR = os.path.join(ROOT_DIR, 'data/ml-100k/')

users = pd.read_csv(os.path.join(DATA_DIR, 'u.user'), 
                        sep='|', header=None, engine='python', encoding='latin-1')

# Nous définissons les différentes variables en fonction de l'information fournie dans le fichier 'readme'
users.columns = ['Index', 'Age', 'Gender', 'Occupation', 'Zip code']

# Bref aperçu
users.head()

Avant de présenter les statistiques descriptives liées à la population étudiée, nous allons dans un premier temps traiter les données associées aux usagers sous la forme d'une <a href="https://en.wikipedia.org/wiki/List_(abstract_data_type)">list</a> afin de pouvoir plus aisément les manipuler. Notons que l'occupation de chaque individu étant une chaîne de caractères, nous avons recodé les 21 occupations possibles en booléen.

In [ ]:
# Nombre d'utilisateurs et d'utilisatrices
nb_users = len(users)

# Sexe
gender = np.where(np.matrix(users['Gender']) == 'M', 0, 1)[0]

# Occupation
occupation_name = np.array(pd.read_csv(os.path.join(DATA_DIR, 'u.occupation'), 
                                            sep='|', header=None, engine='python', encoding='latin-1').loc[:, 0])

# Recodage en booléen de la variable occupation
occupation_matrix = np.zeros((nb_users, len(occupation_name)))

for k in np.arange(nb_users):
    occupation_matrix[k, occupation_name.tolist().index(users['Occupation'][k])] = 1

# Concatenation des différentes données sociodémographiques sous forme de liste
user_attributes = np.concatenate((np.matrix(users['Age']), np.matrix(gender), occupation_matrix.T)).T.tolist()

Nous explorons par la suite les différentes statistiques descriptives associées aux usagers. Celles-ci comportent des informations en lien avec l'âge (variable continue), le sexe (variable binaire) et l'occupation de chacun des usagers (au nombre de 21, toutes binaires).


Statistiques descriptives associées à <i>Age.

In [ ]:
pd.DataFrame(users['Age'].describe()).T

Diagramme à bandes pour la stastistique associée au <i> genre.

In [ ]:
utl.barplot(['Women', 'Men'], np.array([np.mean(gender) , 1 - np.mean(gender)]) * 100, 
            'Sex', 'Percentage (%)', "User's gender", 0)

Diagramme à bandes pour la stastistique associée à <i>Occupation.

In [ ]:
attributes, scores = utl.rearrange(occupation_name, np.mean(occupation_matrix, axis=0) * 100)
utl.barplot(attributes, scores, 'Occupation', 'Percentage (%)', "User's occupation", 90)

### 1.3.2 Movies: Importation et reformatage des données

De la même façon, nous allons traiter et explorer les données associées aux films. Pour chacun d'eux, nous disposons du titre, de la date de sortie en Amérique du Nord, de même que les genres auxquels il est associé.

In [ ]:
movies = pd.read_csv(os.path.join(DATA_DIR, 'u.item'), sep='|', header=None, engine='python', encoding='latin-1')

# Nombre de films
nb_movies = len(movies)

# Genres
movies_genre = np.matrix(movies.loc[:, 5:])
movies_genre_name = np.array(pd.read_csv(os.path.join(DATA_DIR, 'u.genre'), sep='|', header=None, engine='python', encoding='latin-1').loc[:, 0])

# Survol rapide
movies.columns = ['Index', 'Title', 'Release', 'The Not a Number column', 'Imdb'] + movies_genre_name.tolist()
movies.head()

Nous présentons les proportions de films en fonction du genre comme statistique descriptive.

In [ ]:
attributes, scores = utl.rearrange(movies_genre_name, 
                                   np.array(np.round(np.mean(movies_genre, axis=0) * 1, 2))[0])
utl.barplot(attributes, np.array(scores) * 100, xlabel='Genre', ylabel='Percentage (%)', 
            title=" ", rotation = 90)

### 1.3.3 Ratings: Importation et traitement des données

La base de données comportant les évaluations des films effectuées par les usagers est constituée d'environ 100 mille lignes (une évaluation par ligne) où sont respectivement recensés le numéro d'identification de l'utilisateur, le numéro d'identification du film, l'évaluation associée et un marqueur de temps auquel le film a été visionné. Les ensembles d'entraînement et de test ont été fournis tels quels, c'est-à-dire que nous n'avons pas besoin de les construire nous-même, et comportent respectivement 80 et 20 mille évaluations.

Pour des raisons pratiques, nous convertissons la base de données sous la forme d'une liste grâce à la fonction maison convert.

In [ ]:
training_set = np.array(pd.read_csv(os.path.join(DATA_DIR, 'u1.base'), delimiter='\t'), dtype='int')
testing_set = np.array(pd.read_csv(os.path.join(DATA_DIR, 'u1.test'), delimiter='\t'), dtype='int')

train_set = utl.convert(training_set, nb_users, nb_movies)
test_set = utl.convert(testing_set, nb_users, nb_movies)

Comme nous l'avons fait auparavant, nous pouvons obtenir quelques statistiques descriptives associées aux évaluations. Dans une premier temps, il pourrait être intéressant d'étudier les tendances moyennes des individus.

##### Question 1

1. Quelles autres types de statistiques pourraient être intéressantes?

In [ ]:
train_matrix = np.array(train_set)
shape = (len(train_set), len(train_set[0]))
train_matrix.reshape(shape)
train_matrix_bool = np.where(train_matrix > 0 , 1, 0)

user_watch = np.sum(train_matrix_bool, axis=1)
pd.DataFrame(user_watch).describe().T

Avec un petit histogramme...

In [ ]:
sns.set(rc={'figure.figsize':(12,8)})
sns.set(font_scale = 1.5)

plt.title('Empirical distribution of \n the number of movies watched per user')
plt.xlabel('Number of movies watched')
plt.ylabel('Number of users')
plt.hist(user_watch, 100);

Nous présentons finalement quelques statistiques associées aux films.

In [ ]:
movie_frequency = np.mean(train_matrix_bool, axis=0)
pd.DataFrame(movie_frequency).describe().T

##### Question 2

a. Quelles statistiques ou observations pourraient nous paraître pertinentes? Pourquoi?

b. Quel type de statistique pourrait être plus approprié dans un tel contexte?

In [ ]:
plt.xlabel('Proportion of the population who watched the movie')
plt.ylabel('Number of Movies')
plt.hist(movie_frequency, 100);

#### Quel type de statistique pourrait être plus approprié dans un tel contexte?

Nous pourrions également nous intéresser au comportement d'un individu en particulier. Entre autres choses, nous pourrions étudier s'il y a un biais associé à son schème d'évaluation ou encore quelles sont ses préférences cinématographiques en fonction du score attribué.

In [ ]:
def stats_user(data, movies_genre, user_id):
    
    ratings = data[user_id]
    stats = np.zeros(6)
    eva = np.zeros((6, movies_genre.shape[1]))

    for k in np.arange(len(ratings)):
        index = int(ratings[k])
        stats[index] += 1
        eva[index, :] = eva[index, :] + movies_genre[k]

    return stats, eva

In [ ]:
user_id = 0
stats, eva = stats_user(train_set, movies_genre, user_id)
utl.barplot(np.arange(5) + 1, stats[1:6] / sum(stats[1:6]), xlabel='Number of stars', ylabel='Percentage of movies (%)', 
            title=" ", rotation = 0)

##### Question 3

3. Comment vérifier qu'il existe un biais associé au schème d'évaluation d'un individu?

## 1.4 Création des sous-ensembles d'entraînement et de validation

En apprentissage automatique, nous manipulons des <a href="https://blogs.nvidia.com/blog/2018/04/15/nvidia-research-image-translation/">bases de données complexes</a> pour lesquelles nous tentons de définir des espaces de fonctions tout aussi complexes dans le but d'accomplir une tâche précise. Ceci étant, ces espaces de fonctions sont définis par un ensemble de paramètres dont le nombre tend à augmenter avec la complexité des données. Une fois l'espace défini par un ensemble de paramètres fixés, nous pouvons varier les différentes valeurs d'hyperparamètres afin d'explorer empiriquement les espaces de fonctions. Pour choisir l'ensemble des paramètres et d'hyperparamètres optimaux, nous définissons une métrique nous permettant d'évaluer le modèle; par exemple, à quel point l'image d'un chat nous paraît vraisemblable.

Dans la mesure où nous voulons développer un modèle capable de généraliser, l'évaluation de ses performances, et donc la sélection des (hyperparamtètres) doit se faire sur un ensemble de données indépendant, mais issues de la même distribution, de l'ensemble sur lequel il a appris. Pareil ensemble porte le nom d'ensemble de validation.

**! Remarque !** 

La notion d'ensemble d'entraînement et de test dans le cadre de système de recommandation est quelque peu différente de ce que l'on voit habituellement avec les problèmes dits supervisés. Si dans le cadre d'un problème supervisé, l'ensemble de test consiste essentiellement en de nouvelles observations (lire lignes d'un fichier) indépendantes des observations préalablement observées dans l'ensemble d'entraînement, le paradigme est sensiblement différent lorsque nous travaillons avec des systèmes de recommandation.

Effectivement, et en raison du modèle mathématique sur lequel est basé les systèmes de recommandation, les données appartenant à l'ensemble de test ne sont pas liées à un nouvel individu, mais bien à de nouvelles évaluations, faites par le même ensemble d'individus, mais jusqu'alors inobservées. Dès lors, les données associées aux ensembles d'entraînement, de validation et de test ne sont plus indépendantes tel que supposé (la fameuse hypothèse iid ); ce qui complique théoriquement les choses.

Puisque le but de l'atelier n'est pas d'étudier la notion de biais associée au type de dépendance entre les différents évaluations dans les systèmes de recommendation, nous allons naïvement supposer que chacune des évaluations sont indépendantes les unes des autres. N'empêche, dans un cadre pratique, ignorer ce genre de considérations pourra éventuellement biaiser les algorithmes.

In [ ]:
def split(data, ratio, tensor=False):
    train = np.zeros((len(data), len(data[0]))).tolist()
    valid = np.zeros((len(data), len(data[0]))).tolist()

    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] > 0:
                if np.random.binomial(1, ratio, 1):
                    train[i][j] = data[i][j]
                else:
                    valid[i][j] = data[i][j]

    return [train, valid]

train_0 = split(train_set, 0.8)
test = test_set

# 2\. Systèmes de recommandation : factorisation matricielle

## 2.1 Modèle

La factorisation matricielle suppose que chaque évaluation observée $r\_{ui}$ pour $1 \\leq u \\leq \|U\|$ et $1 \\leq i \\leq \|I\|$, dans lesquelles $\|U\|$ et $\|I\|$ représentent respectivement le nombre d’utilisateurs et le nombre de films, peut être estimée à l’aide d’un modèle latent. Ce modèle présente l’estimation $\\hat{r}_{ui}$ de l’évaluation observée $r_{ui}$ ainsi :

$$ \\begin{align} \\hat{r}_{ui} =  \\langle p_{u}, q\_{i} \\rangle, \\end{align} $$

dans laquelle $\\langle \\cdot \\rangle$ est le produit scalaire et $p\_{u}$ et $q\_{i}$ sont les représentations latentes associées à l’utilisateur *u* et à l'item *i*. L’intuition derrière cette représentation suggère que chaque évaluation peut être estimée en considérant une caractérisation latente des utilisateurs et des items.

Par exemple, déterminons qu’il y a 3 variables latentes et supposons qu’elles sont associées respectivement à la popularité des films selon le box-office, à leur durée et à leur degré de romance. Supposons que l’utilisateur *u* est un adolescent de 15 ans qui aime les films d’horreur populaires d’assez courte durée : $$ \\begin{align} p\_{u} = \[1, 0, 0]\^T. \\end{align} $$

Supposons maintenant que le film *i* s'avère être *Le Roi Lion* selon la modélisation latente suivante :

$$ \\begin{align} q\_{i} = \[1, 0.5, 0]\^T. \\end{align} $$

L’estimation de l’évaluation pour cet utilisateur et cet élément en fonction des représentations latentes sera donc :

$$ \\begin{align} \\hat{r}\_{ui} =  \\langle p\_u, q\_i \\rangle = 1. \\end{align} $$

Le défi principal pour ce type de modèle est de définir l’ensemble des vecteurs latents associés aux utilisateurs, regroupés dans la matrice $\\mathbf{P}_{\|U\| \\times k} = \[p\_1, p\_2, .. ., p\_k]$, et aux éléments, regroupés dans la matrice $ \\mathbf{Q}_{\|I\| \\times k} = \[q\_1, q\_2, ..., q\_k] $.   Puisque le problème initial est de présenter les estimations les plus exactes, donc de calculer $\\mathbf{P}$ et $\\mathbf{Q}$ de façon à minimiser les différences entre la totalité des évaluations observées $r\_{ui}$ et leur estimation $\\hat {r}\_{ui}$, nous pouvons définir la tâche à accomplir avec le problème d’optimisation suivant :

$$ \\begin{align} \\mathbf{P}, \\mathbf{Q} = \\underset{p, q}{\\operatorname{argmin}} \\sum\_{r\_{ui} \\neq 0} (r\_{ui} - \\hat{r}_{ui})\^2 = \\underset{p, q}{\\operatorname{argmin}}  \\sum_{r\_{ui} \\neq 0} (r\_{ui} - \\langle p\_u, q\_i \\rangle)\^2. \\end{align} $$

Nous pouvons aussi régulariser les variables latentes dans le but de forcer les vecteurs associés à présenter des valeurs autres que 0 :

$$ \\begin{align} \\mathbf{P}, \\mathbf{Q} = \\underset{p, q}{\\operatorname{argmin}} \\sum\_{r\_{ui} \\neq 0} {(r\_{ui} - \\langle p\_u, q\_i \\rangle)\^2 + \\lambda(\|\|p\_u\|\|\^2 + \|\|q\_i\|\|\^2)}, \\end{align} $$

où $\\lambda$ est l’hyperparamètre de régularisation, soit la <i>pénalisation des poids</i> en apprentissage profond et le multiplicateur de Lagrange en mathématiques. Notez que les vecteurs latents avec très peu de valeurs nulles prédiront en retour des évaluations d’une valeur autre que 0. Puisque nous essayons de proposer de nouvelles recommandations, cette contrainte semble utile pour éviter une estimation sous la forme d’une matrice creuse $\\hat{\\mathbf{R}}$.

En général, ce problème d’optimisation consistant à factoriser une matrice creuse ne peut pas être résolu aussi facilement qu’un problème de régression linéaire, entre autres, à l’aide de la méthode des moindres carrés. Dans ce tutoriel, nous introduirons l’algorithme de descente de gradient stochastique dans une approche visant à résoudre le problème d'optimisation pour estimer les matrices $\\mathbf{P}$ et $\\mathbf{Q}$.


## 2.2 Implémentation

Afin de concevoir un système de recommandation basé sur la factorisation matricielle, il faut déterminer quelques fonctions spécifiques requises pour ce type d’algorithme. En gros, l’implémentation se divise en trois composantes :

1. **La boucle d’entraînement** : processus d’optimisation itératif permettant d’estimer à quel point le modèle n’arrive pas à satisfaire un objectif spécifique et de faire les adaptations nécessaires au modèle jusqu’à ce que le critère d’arrêt soit atteint.

2. **La fonction de perte** : fonction calculant à quel point les prédictions du modèle diffèrent des observations réelles.

3. **L’estimation** : processus d’estimation des matrices des facteurs $\\mathbf{P}$ et $\\mathbf{Q}$ associés respectivement avec les utilisateurs et les éléments.


### 2.2.1 Boucle d’entraînement

Configurons maintenant la boucle d’entraînement. Pour cela, nous utilisons une fonction qui effectue un certain nombre d’itérations pour actualiser les paramètres de notre modèle jusqu’à ce que le critère d'arrêt soit atteint.


##### Question 4

Supposons que nous avons le choix entre la fonction de prédiction, la descente de gradient stochastique et la fonction de perte. Nous sauterons l’implémentation de ces fonctions pour se concentrer sur leur signature (vous pouvez jeter un coup d’œil aux cellules de code ci-dessous).

1. Dans le cas des estimateurs obtenus via la descente de gradient stochastique (SGD), quelle condition devrions-nous rajouter à la ligne 17? Et pourquoi?
2. À la fin de chaque époque, quelle statistique serait-il préférable de calculer? Codez-la. Remarque : il est préférable d'initialiser des objets en début de fonction (voir ligne 6).
3. Donnez deux raisons pour lesquelles ces statistiques sont utiles.
4. Le critère d'arrêt pour éviter le surapprentissage à la ligne 30 est plutôt naïf. Pourquoi?
5. Développez un nouveau critère d'arrêt.


In [ ]:
def learn_to_recommend(data, features=10, lr=0.0002, epochs=101, weigth_decay=0.02, stopping=0.001):
       """
    Args:
      data: ensemble des évaluations
      features: variables latentes
      lr: learning rate pour la descente de gradient
      epochs: nombre d'iterations ou boucles maximales à effectuer
      weigth_decay: régularisation de type L2 afin de predire des valeurs differentes de 0
      stopping: scalaire associé au critère d'arrête
      
    Returns:
      P: matrice latente associée aux usagers
      Q: matrice latente associée aux items
      loss_train: un vecteur des différentes valeurs de la fonction perte après chaque itération sur train
      loss_valid: un vecteur des différentes valeurs de la fonction perte après chaque itération pas sur valid
      """
     
    train, valid = data[0], data[1]
    nb_users, nb_items = len(train), len(train[0])

    # Question 4.2: Listes à initialiser 
    # 

    P = np.random.rand(nb_users, features) * 0.1
    Q = np.random.rand(nb_items, features) * 0.1
    
    for e in range(epochs):        
        for u in range(nb_users):
            for i in range(nb_items):

                # Question 4.1: Codez la condition sur la ligne ci-dessous.
                # if ...
                    error_ui = train[u][i] - prediction(P, Q, u, i)
                    P, Q = sgd(error_ui, P, Q, u, i, features, lr, weigth_decay)
                               
       # Question 4.2: Codez la statistique
        #
        #
        
        if e % 10 == 0:
            print('Epoch : ', "{:3.0f}".format(e+1), ' | Train :', "{:3.3f}".format(loss_train[-1]), 
                  ' | Valid :', "{:3.3f}".format(loss_valid[-1]))

        # Question 4.4: De la naïveté du critère d'arrêt
        if abs(loss_train[-1]) < stopping:
            break
            
        # Question 4.5 : Nouveau critère d'arrêt
        # if ... :
            break
        
    return P, Q, loss_train, loss_valid

### 2.2.2 Fonction de coût

La fonction de coût joue un rôle déterminant dans la construction d'un modèle prédictif. En effect, c'est cette même fonction de coût que nous essaierons de minimiser (ou maximiser c'est selon) en ajustant de façon itérative les valeurs des matrices latentes.

Dans la mesure où l'on considère que les évaluations considérées varient entre 1 et 5, l'erreur quadratique moyenne (EQM) semble une première option intéressante. Formellement, dans le cadre d'un système de recommandation, nous définierons l'EQM ainsi :

$$ \\begin{align} MSE(\\mathbf{R}, \\hat{\\mathbf{R}}) = \\frac{1}{n} \\sum\_{r\_{ui} \\neq 0} (r\_{ui} - \\hat{r}\_{ui})\^2, \\end{align} $$

où $\\mathbf{R}$ et $\\hat{\\mathbf{R}} $ sont respectivement les matrices des évaluations observées et prédites et *n* représente le nombre d'évaluations. De la même façon, $r_{ui}$ et $\hat{r}_{ui}$ sont des scalaires associés respectivement à l'évaluation observée et l'évaluation estimée de l'usager $u$ pour l'item $i$.


##### Question 5

1. Supposons que nous voulons prédire l'évaluation de l'individu <i>i</i> pour le film <i>j</i>, comment devrions-nous nous y prendre? Implémentez la fonction prédiction.
2. Un détail important est manquant dans la fonction `loss` suivante. En quoi cette erreur est fondamentale? Corrigez-la. 


In [ ]:
# TODO 5.1:
def prediction(P, Q, u, i):
    """
    Args:
       P: matrice des usagers
       Q: matrice des items
       i: indice associé à l'usager i
       j: indice associé à l'item j
    
    Returns:
       pred: l'évaluation prédite de l'usager i pour l'item j
       """
    pass

def loss(data, P, Q):
    """
    Args:
       data: données
       P: matrice des usagers
       Q: matrice des items
       
    Returns:
        EQM: la moyenne observée des erreurs au carré
        """
        
    errors_sum, nb_evaluations = 0., 0
    nb_users, nb_items = len(data), len(data[0])

    for u in range(nb_users):
        for i in range(nb_items):
        
            # Question 5.2: Un détail important
            #
                errors_sum += pow(data[u][i] - prediction(P, Q, u, i), 2)
                nb_evaluations += 1
                
    return errors_sum / nb_evaluations

### 2.2.3 Estimation

La méthode d'estimation des paramètres du modèle est directement associée à la fonction de coût que nous essayons de minimiser. Avec la factorisation matricielle, deux techniques d'estimation sont disponibles afin de calculer les matrices latentes $\mathbf{P}$ et $\mathbf{Q}$ respectivement associées aux usagers et aux items. Dans tous les cas, ces techniques font appel à la linéarité du modèle de factorisation matriciel.

#### Descente de gradient

Dans un premier temps, nous implémentons la descente stochastique du gradient (SGD): une méthode itérative passant en revue l'ensemble des évaluations non nulles pour chacun des usagers.

Formellement, et en se rappelant que la fonction que nous essayons de minimiser est:

$$ \\begin{align} \\underset{p, q}{\\operatorname{min}} L(\\mathbf{R}, \\lambda) = \\underset{p, q}{\\operatorname{min}} \\sum\_{r\_{ui} \\neq 0} {(r\_{ui} - \\langle p\_u, q\_i \\rangle)\^2 + \\lambda \\cdot (\|\|p\_u\|\|\^2 + \|\|q\_i\|\|\^2)}, \\end{align} $$

nous calculons que les gradients de la précédente équation en fonction de $p_u$ et $q_i$ sont:

$$ \\nabla\_{p\_{u}} L(\\mathbf{R}, \\lambda) =  -2q\_{i} \\cdot \\epsilon\_{ui} + 2\\lambda \\cdot p\_{u} \\quad \\text{and} \\quad \\nabla\_{q\_{i}} L(\\mathbf{R}, \\lambda) =  -2p\_{u} \\cdot \\epsilon\_{ui} + 2\\lambda \\cdot q\_{i}, $$

où

$$ \\epsilon\_{ui} = r\_{ui} - \\hat{r}\_{ui}. $$

Enfin, pour chaque itération, et dans la mesure où l'évaluation observée est non-nulle, chacune des mises à jours des vecteurs latentes pourra se faire ainsi:

$$
p_{u}^{(t+1)} \leftarrow p_{u}^{(t)} + \gamma \cdot (\epsilon_{ui} \cdot q_{i}^{(t)} - \lambda \cdot p_{u}^{(t)}) \\
q_{i}^{(t+1)} \leftarrow q_{i}^{(t)} + \gamma \cdot (\epsilon_{ui} \cdot p_{u}^{(t)} - \lambda \cdot q_{i}^{(t)}),
$$

où $p_{u}^{(t+1)}$ dénote la valeur de $p_{u}$ après la $t + 1$ ième itération et où $\gamma$ est le pas d'apprentissage (<i>learning rate</i>) de la descente. 

#### Remarque sur les moindres carrés alternés

La deuxième technique est basée sur les moindres carrés alternés (ALS). Cette méthode a ceci d'élégant qu'elle permet une forme analytique. Nous ne l'implémenterons pas dans le cadre de cet atelier.


##### Question 6

1. Compte tenu des équations ci-dessus, pouvez-vous compléter la fonction `sgd` ci-dessous afin de mettre à jour les paramètres $\\mathbf{P}$ et $\\mathbf{Q}$ de notre modèle?


In [ ]:
def sgd(error, P, Q, id_user, id_item, features, weight_decay, lr):
    """
    Args:
       error: différence entre l'évaluation observée et celle prédite (dans cet ordre)
       P: matrix of users
       Q: matrix of items
       id_user: id_user
       id_item: id_item
       features: nombre de variables latente
       lr: pas d'apprentissage pour la descente du gradient
       
    Returns:
        P: la nouvelle estimation de P
        Q: la nouvelle estimation de Q
     """
    
    # for :
        #
        #
    
    return P, Q

## 2.3 Entraînement du modèle

La factorisation matricielle maintenant implémentée, nous pouvons commencer à entraîner le modèle avec différents paramètres et hyperparamètres. L'idée ici n'est pas d'ajuster les paramètres de façon telle à obtenir le meilleur modèle possible, mais simplement de comprendre le rôle que ceux-ci peuvent jouer, tant du point de vue du surappentissage que du temps de calcul. En fait, ici, il n'y a que très peu de mauvaises réponses.


In [ ]:
features = 5
lr = 0.02
epochs = 101
weigth_decay = 0.02
stopping = 0.01

P, Q, loss_train, loss_valid = learn_to_recommend(train, features, lr, epochs, weigth_decay, stopping)

Une fois le modèle entraîné, nous pouvons visualiser les différentes courbes d'apprentissage.


In [ ]:
x = list(range(len(loss_train)))
k=0

sns.set(rc={'figure.figsize':(12,8)})
sns.set(font_scale = 1.5)

plt.plot(x[-k:], loss_train[-k:], 'r', label="Train")
plt.plot(x[-k:], loss_valid[-k:], 'g', label="Validation")
plt.title('Learning curves')
plt.xlabel('Epoch')
plt.ylabel('MSE')
leg = plt.legend(loc='best', shadow=True, fancybox=True)

##### Question 7

1. Était-ce vraiment nécessaire de calculer autant d'époques?
2. En nous inspirant de ces courbes, quel critère d'arrêt pourrions-nous développer?
3. En quoi est-il plus pertinent que celui défini dans la boucle d'apprentissage?
4. Implémentez-le.


In [ ]:
# TODO 7.4

Enfin, nous pouvons évaluer les performances de notre modèle sur l'ensemble test.

##### Question 8

1. Implémentez la procédure.
2. En quoi est-ce pertinent d'évaluer les performances sur un tel ensemble?


In [ ]:
# TODO 8.2


## 2.4 Analyses


### 2.4.1 Exploration des couches latentes

Grâce à la factorisation matricielle, il est possible d'explorer les différentes variables latentes associées aux usagers et aux items. De par la nature des matrices $\mathbf{P}$ et $\mathbf{Q}$, l'exploration des <i>k</i> variables latentes associées aux colonnes de $\mathbf{P}$ et $\mathbf{Q}$ pourraient s'avérer intéressante. 

À titre d'exemple, supposons que les deux premières variables latentes de la matrice des objets $\mathbf{Q}$ présentent les valeurs suivantes: 

$$
\begin{align}
q_1 &= [-1.0, \ -0.8, \ 0.0, \ ..., \ 1.0, \ 0.5 ] 
\qquad \text{et} \qquad
q_2 = [-1.0, \ 0.8,  \ 1.0, \ ..., \ 0.5, \ -0.8 ].
\end{align}
$$

Enfin, supposons qu'à ces valeurs correspondent les films suivant: 

1. The Room (2003),
2. Star Wars: Attack of the clones (2002),
3. Titanic (1997),
4. Citizen Kane (1954),
5. The Nigthmare before Christmass (1993).

En cartographiant ces films en fonctions des valeurs associées des deux premiers variables latentes, nous obtenons le graphique suivant:

<img src = "https://user-images.githubusercontent.com/13997178/91663551-9486d300-eab7-11ea-8e9f-c58398eff9fe.png" width = "500">

Ce graphe permet possiblement de se faire une id&eacute;e des sch&eacute;mas sous-tendant chaque variable latente. Dans ce cas, on pourrait s&rsquo;imaginer que la premi&egrave;re variable latente est associ&eacute;e &agrave; comment le film est re&ccedil;u et que la deuxi&egrave;me est reli&eacute;e &agrave; la pr&eacute;sence d&rsquo;une grande vedette. Du moins, c&rsquo;est une hypoth&egrave;se int&eacute;ressante! Voyons s&rsquo;il y a des tendances similaires lorsqu&rsquo;on s&rsquo;int&eacute;resse aux valeurs associ&eacute;es &agrave; la matrice des utilisateurs $ \mathbf{P}$. Supposez que les deux premi&egrave;res variables latentes de la matrice utilisateur $\mathbf{P}$ prennent les valeurs suivantes&nbsp;: 

$$ \begin{align} p_1 &amp;= [1.0, \ 0.0, \ -0.5, \ 1.0, \ -1.0, \ ...] \qquad \text{and} \qquad p_2 = [1.0, \ 0.0, \ 0.5, \ -1.0, \ -0.8, \ ...] \end{align} $$ 

Et supposez que chaque valeur dans ces deux ensembles correspond aux utilisateurs ci-dessous&nbsp;: 

1. Serena 
2. Kali 
3. Neil 
4. Mary 
5. David 

Relions maintenant les utilisateurs aux valeurs associ&eacute;es aux vecteurs $ p_1 $ et $ p_2 $. Remarquez que dans ce cas, nous avons consid&eacute;r&eacute; les deux m&ecirc;mes facteurs latents afin de pouvoir les comparer &agrave; la caract&eacute;risation des axes obtenus ant&eacute;rieurement&nbsp;: 

<img src = "https://user-images.githubusercontent.com/13997178/91663557-9e103b00-eab7-11ea-9ee1-0de6a5ac3760.png" width = "500">

Cette approche pourrait nous permettre de recommander de nouveaux films n&apos;ayant jamais &eacute;t&eacute; &eacute;valu&eacute;s par les utilisateurs simplement en se fiant &agrave; certaines caract&eacute;ristiques. Par exemple, il y a de grandes chances que Serena aime le prochain film de Scorsese <i>The Irishman</i> et que Neil ait h&acirc;te de visionner le nouveau film <i>Cats</i>. Nous proposons maintenant une fonction facilitant l&apos;exploration des variables latentes.


In [ ]:
def exploration(movie_titles, latent_matrix, frequency_mask, factor_idx, k):
    """
    Args:
       object_name: vecteur des noms des films
       matrix: matrice latente associée aux films
       freq: seuil de fréquence de visionnement minimal au-delà duquel nous pouvons considérer le film dans l'analyse
       factor: le numéro de la variable latente que l'on veut étudier
       k: nombre de films en sortie = 3*k - 1
       
    Returns:
        names: le titre des films
        scores: l'évaluation prédite associée
    """

    # slice the column to obtain latent variable, then apply mask
    latent_variable = latent_matrix[:, factor] * frequency_mask

    # filter out infrequent movies
    nonzero_indices = np.nonzero(latent_variable)
    movies = np.array(movie_titles)[nonzero_indices][:k]
    latent_variable = latent_variable[nonzero_indices][:k]

    return movies, latent_variable


Utilisons cette fonction et visualisons les résultats. Pour ce faire, nous considérerons les films ayant été visionnés par au moins 10 $\%$ de tous les utilisateurs. Utilisons la liste movie\_popularity créée plus tôt.


In [ ]:
# print(movie_popularity)
# print(movie_popularity.shape)

In [ ]:
k = 2
factor = 0
threshold = 0.1
names, scores = exploration(movies['Title'], Q, np.where(movie_frequency > threshold, 1, 0), factor, k)

df = pd.DataFrame(np.matrix((names, scores)).T, (np.arange(len(scores)) + 1).tolist())
df.columns = ['Title', 'Latent factor']
df

##### Question 9

1. Est-ce que certaines variables latentes peuvent être interprétables?
2. Qu'arrivera-t-il si nous augmentons le nombre de variables latentes? Si nous le diminuons?


# 3\. Applications

L'un des objectifs premier des systèmes de recommandation est d'effectuer de recommandations (!) personnalisées pour chacun des utilisateurs. Dès lors, il pourrait être intéressant d'étudier les recommandations effectuées par notre modèle pour un individu spécifique. Naturellement, les recommendations faites ne suggèrent que des films non visionnés par l'usager.




##### Question 10

1. Nous allons maintenant, pour un usager choisi, effectuer les dix meilleures recommandations associées. Pour ce faire, nous allons procéder par étapes simples tel que présentées dans le code. Notez que la fonction maison `rearrange` présentée ci-dessous pourrait vous être utile.


In [ ]:

def rank_top_k(names, ratings, k=10):
   """
   Example:
   a, b = np.array(['a', 'b', 'c']), np.array([6, 1, 3])
   a, b = rank_top_k(a, b, k=2)
   >>> a
   np.array('a', 'c')
   >>> b
   np.array([6, 3])
   """
 
   # rank indices in descending order
   ranked_ids = np.argsort(ratings)[::-1]
 
   return names[ranked_ids][:k], ratings[ranked_ids][:k]

In [ ]:
user_id = 0
top_k = 10

# Étape 1: Définir quels films ont déjà été visionnés dans chacun des sous ensembles

# Étape 2: Calculez l'ensemble de évaluations prédites pour l'individu choisi

# Étape 3: Considérez seulement les évaluations associées aux ensembles d'entraînement et de validation

# Étape 4: Réorganisez les estimations sur les différents ensembles de manière à proposer les 10 meilleures
#          recommandations

# Présentations des recommandations





En manipulant rapidement le code ci-dessus, on s'aperçoit que les évaluations sur les prédictions sur les ensembles d'entraînement, de validation et même de test sont vraisemblables. Effectivement, ces dernières se situent entre 0 et 5, tout semble alors en ordre, ce qui nous paraît "normal" puisque les EQM sur les ensembles d'entraînement et de validation n'étaient pas particulièrement élevées.

En fait, ça pourrait être intéressant de proposer à l'usager des films en fonction de ces préférences du moment ou en fonction du genre.

##### Question 11

1. Écrivez une fonction permettant d'effectuer pareille tâche.


In [ ]:
def recommend(user_id, data, P, Q, list_of_genre_names, movies_genre, genre):
    """
    Args:
       user_id: user_id
       data: ensemble des évaluations
       P: matrice des invidus
       Q: matrices des items
       movies_genre: genre de film que l'utilisateur user_id veut écouter
       new: Booléen, est-ce que nous voulons effectuer des nouvelles recommendations ou pas?
    
    Returns:
        les meilleures suggestions en fonction du genre de film sélectionné
    """ 
    
    # ...

    return np.array(predictions) * np.array(genre.T)[0]

In [ ]:
# print(movies_genre_name)
# print(movies_genre.shape)

In [ ]:
genre = "Animation"
user_id = 1
top_k = 5
 
# Calcul et réorganisation des recommandations
estimates = recommendations(user_id, train, P, Q, list_of_genre_names=movies_genre_name, movies_genre=movies_genre, genre=genre)
 
recommendations, scores = rank_top_k(np.array(movies['Title']), estimates, k=top_k)
 
# Presentation
df = pd.DataFrame(np.matrix((recommendations, scores)).T, (np.arange(top_k) + 1).tolist(), columns = ['Title', 'Predicted rating'])
df